In [96]:
import pandas as pd
import xgboost as xgb
import numpy as np
def AggregationData(ARMA,col):
    ARMA = ARMA.T  
    base = PreProcessing(0,ARMA[0],col)
    for x in range(1,2001):
        k = x-1
        if k == 0:
            continue;
        shop = ARMA[k]
        shop =PreProcessing(k,shop,col)
        base = pd.merge(base,shop,how='outer')
    return base    
def PreProcessing(x,shop,col):
    shop = pd.DataFrame(shop)
    shop.drop('shop_id',inplace=True)
    shop.rename(columns={x:col},inplace='True')
    shop['shop_id'] = x+1
    return shop
def xgb_eval_custom_r(y_pred, dtrain):
    y_true = dtrain.get_label()
    up = np.sum(y_pred - y_true)
    down = np.sum(y_pred + y_true)
    whole = np.abs(up/down)
    Loss = whole/len(y_true)
    #here only predict one shop 
    return 'Loss', Loss

In [97]:
ARMA_D = pd.read_csv('C:\Users\Administrator\PycharmProjects\Customer\Predict\NewData\ARMA_D.csv')
ARMA = pd.read_csv('C:\Users\Administrator\PycharmProjects\Customer\Predict\NewData\ARMAmissing.csv')
ARMA = AggregationData(ARMA,'ARMA')
ARMA_D = AggregationData(ARMA_D,'ARMA_D')

In [98]:
ARMA['ARMA_D'] = ARMA_D['ARMA_D']
ARMA['ARMA_D_id'] = ARMA_D['shop_id']

In [99]:
ARMA

,ARMA,shop_id,ARMA_D,ARMA_D_id
0,212,1,158,1
1,221,1,164,1
2,223,1,166,1
3,223,1,166,1
4,224,1,166,1
5,224,1,166,1
6,224,1,166,1
7,224,1,166,1
8,224,1,166,1
9,224,1,166,1


In [ ]:
ToDrop=['Num','shop_id']
train_xgb = xgb.DMatrix(data=train_df.drop(ToDrop,axis=1)
                        ,label=train_df['Num'])
valid_xgb = xgb.DMatrix(data=test_df.drop(ToDrop,axis=1)
                        ,label=test_df['Num'])
predict_xgb = xgb.DMatrix(data = predict_df.drop(ToDrop,axis = 1))
params = {
  'objective': 'reg:linear'
  ,'eta': 0.1
  ,'max_depth': 6
  , 'subsample': 0.4
  , 'colsample_bytree': 0.9
  ,'min_child_weight': 12
  ,'gamma': 0.07
  , 'seed': 10
,'reg_alpha': 0.06
}       
evallist = [(train_xgb, 'train'), (valid_xgb, 'valid')]
model = xgb.train(params.items()
                  , dtrain=train_xgb
                  , num_boost_round=10000
                  , evals=evallist
                  , early_stopping_rounds=20
                  , maximize=False
                  , verbose_eval=10
                  , feval=xgb_eval_custom_r
              )
print "now it is:",shop_id
print ('get info from model')                  
print (model.best_score, model.best_iteration,model.best_ntree_limit)

result = pd.DataFrame(model.predict(predict_xgb,ntree_limit=model.best_ntree_limit))  
result = result.rename(columns={0:'Num'})
each_line = []
each_line.append(shop_id)
for Num in result.Num:
    each_line.append(Num) 
p_result.append(each_line)  